In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data from csv file here: "S:\Zack\Imagery\Hazelnut\WPP_Farm_LiDAR\ArcGIS_Pro_WPP_Farm_LiDAR\WPP_BlockB_Hazelnut_CrossDim_HW.csv"
data = pd.read_csv("S:\Zack\Imagery\Hazelnut\WPP_Farm_LiDAR\ArcGIS_Pro_WPP_Farm_LiDAR\WPP_Farm_BlockB_LiDAR_Canopies_VertexPointCoords.csv")

In [2]:
# list columns in data
print(data.columns)

Index(['treeID', 'area', 'diameter', 'volume', 'xmin', 'ymin', 'xmax', 'ymax',
       'width', 'length', 'POINT_X', 'POINT_Y'],
      dtype='object')


In [3]:
# width = xmax - xmin
# length = ymax - ymin
new_data = data.copy()

# only keep the first row of each unique treeID
new_data = new_data.drop_duplicates(subset='treeID')

# drop POINT_X and POINT_Y columns
new_data = new_data.drop(columns=['POINT_X', 'POINT_Y'])

# print first 5 rows
print(new_data.head())


       treeID       area  diameter     volume           xmin          ymin  \
0          99   5.130681  2.421100   3.715833  396328.834638  4.437270e+06   
5649      100   5.971604  2.594172   3.930869  396338.166083  4.437270e+06   
12188     102   4.266943  2.180919   2.739646  396343.148897  4.437270e+06   
16851     104   8.123554  2.983567   5.165697  396348.733156  4.437270e+06   
25295     107  20.194204  3.682439  19.641967  396358.280742  4.437269e+06   

                xmax          ymax     width   length  
0      396331.984978  4.437273e+06  3.150340  2.86119  
5649   396341.387666  4.437273e+06  3.221583  3.55106  
12188  396346.344565  4.437273e+06  3.195668  2.76434  
16851  396353.128801  4.437273e+06  4.395645  3.65281  
25295  396364.835206  4.437274e+06  6.554464  4.25103  


In [4]:
# calculate euclidean width, length, and diameter for each tree
new_data.loc[:, 'diagonal_width'] = np.nan
new_data.loc[:, 'diagonal_length'] = np.nan
new_data.loc[:, 'diameter'] = np.nan

for treeID in new_data['treeID']:
    tree = data[data['treeID'] == treeID].copy()
    tree.loc[:, 'x_ymin'] = min(tree.loc[tree['POINT_Y'] == tree['POINT_Y'].min(), 'POINT_X'])
    tree.loc[:, 'x_ymax'] = max(tree.loc[tree['POINT_Y'] == tree['POINT_Y'].max(), 'POINT_X'])
    tree.loc[:, 'y_xmin'] = min(tree.loc[tree['POINT_X'] == tree['POINT_X'].min(), 'POINT_Y'])
    tree.loc[:, 'y_xmax'] = max(tree.loc[tree['POINT_X'] == tree['POINT_X'].max(), 'POINT_Y'])
    tree.loc[:, 'diagonal_width'] = np.sqrt(((tree['xmax'] - tree['xmin'])**2 + (tree['y_xmax'] - tree['y_xmin'])**2))
    tree.loc[:, 'diagonal_length'] = np.sqrt(((tree['ymax'] - tree['ymin'])**2 + (tree['x_ymax'] - tree['x_ymin'])**2))

    # find diameter (this takes a while to execute)
    max_distance = 0
    for i in range(len(tree)):
        for j in range(i+1, len(tree)):
            # diameter is longest euclidean distance between any two points
            distance = np.sqrt((tree.iloc[i]['POINT_X'] - tree.iloc[j]['POINT_X'])**2 + (tree.iloc[i]['POINT_Y'] - tree.iloc[j]['POINT_Y'])**2)
            if distance > max_distance:
                max_distance = distance

    new_data.loc[new_data['treeID'] == treeID, 'diameter'] = max_distance
    new_data.loc[new_data['treeID'] == treeID, 'diagonal_width'] = tree['diagonal_width'].values[0]
    new_data.loc[new_data['treeID'] == treeID, 'diagonal_length'] = tree['diagonal_length'].values[0]
    new_data.loc[new_data['treeID'] == treeID, 'x_ymin'] = float(tree['x_ymin'].values[0])
    new_data.loc[new_data['treeID'] == treeID, 'x_ymax'] = float(tree['x_ymax'].values[0])
    new_data.loc[new_data['treeID'] == treeID, 'y_xmin'] = float(tree['y_xmin'].values[0])
    new_data.loc[new_data['treeID'] == treeID, 'y_xmax'] = float(tree['y_xmax'].values[0])

In [5]:
print(new_data.head())

       treeID       area  diameter     volume           xmin          ymin  \
0          99   5.130681  3.299852   3.715833  396328.834638  4.437270e+06   
5649      100   5.971604  3.791410   3.930869  396338.166083  4.437270e+06   
12188     102   4.266943  3.326612   2.739646  396343.148897  4.437270e+06   
16851     104   8.123554  4.397433   5.165697  396348.733156  4.437270e+06   
25295     107  20.194204  6.725250  19.641967  396358.280742  4.437269e+06   

                xmax          ymax     width   length  euclidean_width  \
0      396331.984978  4.437273e+06  3.150340  2.86119         3.152351   
5649   396341.387666  4.437273e+06  3.221583  3.55106         3.335907   
12188  396346.344565  4.437273e+06  3.195668  2.76434         3.326612   
16851  396353.128801  4.437273e+06  4.395645  3.65281         4.397433   
25295  396364.835206  4.437274e+06  6.554464  4.25103         6.558251   

       euclidean_length         x_ymin         x_ymax        y_xmin  \
0              

In [6]:
# save "S:\Zack\Imagery\Hazelnut\WPP_Farm_LiDAR\ArcGIS_Pro_WPP_Farm_LiDAR\WPP_BlockB_Hazelnut_Euclidean_HW.csv"
new_data.to_csv("S:\Zack\Imagery\Hazelnut\WPP_Farm_LiDAR\ArcGIS_Pro_WPP_Farm_LiDAR\outputs_forNate\WPP_BlockB_Hazelnut_LiDAR_Euclidean_HW.csv", index=False)